<a href="https://colab.research.google.com/github/Goodbam/Changing-Naver-rising-searches/blob/master/crawling(12_5_1PM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
!pip install lxml
!pip install parse

In [0]:
import requests                 # url request
from lxml.html import parse     # parsing the data as the html form
from io import StringIO         # String I/O module

THRESHOLD = 28 #limit to check whether they are relative or not
NOISE = 9

In [64]:
url = "https://www.naver.com"

# Get html source
text = requests.get(url).text

# parsing the data as the html form
text_source = StringIO(text)    # read the data as string
parsed = parse(text_source)     # source ->parsing the data as the html form 

#find root node
doc = parsed.getroot()

# doc.findall(".//TagName") => find the tag lists
uls = doc.findall('.//ul')

#Get the Rising searches
ul = uls[10]   
spans = ul.findall('.//span')

data_list= []  # to save all the keywords

cnt = 1
for data in spans:
  if cnt % 2 == 0:  
    print(int((cnt+1)/2),'->',data.text_content().strip())  # show the existing rising searches
  cnt +=1  
  data_list.append(data.text_content().strip()) #save each keyword
    


1 -> 백석역
2 -> 이일재
3 -> 마녀공장
4 -> 이설
5 -> 수능 성적표
6 -> 최유정 변호사
7 -> 2019 수능 만점자
8 -> 광주형 일자리
9 -> pmc 더벙커
10 -> 이선정
11 -> 황지만
12 -> 이강인
13 -> 블레이드앤소울 레볼루션
14 -> 다나
15 -> 나쁜형사
16 -> 모털 엔진
17 -> 홍현희
18 -> 한국교육과정평가원
19 -> 삼성전자 특별보너스
20 -> 기흥 롯데프리미엄 아울렛


In [65]:
path='https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query=' #This is the url for surfing

matrix = [] #to count the score of relationship
label = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] #disjoint set

cnt1 = 0

for i in range(0, 20):
  word = data_list[1 + cnt1]
  search = path + word #url searching the keyword
  text = requests.get(search).text # no worry about encoding url
  
  row = []
  cnt2 = 0
  for j in range(0, 20):
    if cnt1 != cnt2: #no need to check when they are equivalent
      target = data_list[1 + cnt2];
      row.append(text.count(target) + 7*word.count(target) - NOISE) 
      
      if row[int(cnt2/2)] >= THRESHOLD: #IF they are relative,
        print(word, ' and ', target,)
        #the representive number should be minimized
        
        #여기 정규화를 해서 row의합으로 각 row의 값을 나누던지 해보자
        #그러려니, 전체적으로 점수 자체가 높은 것은 감안해서 평균을 다시 루트씌우고 곱하든지 고려해보자
        #A와 B 관련 , B C 관련 이면 A C는 관련이 있는건가? 이것도 감안
        
        if label[int(cnt1/2)] < label[int(cnt2/2)]:
          label[int(cnt2/2)] = label[int(cnt1/2)]  
        
        else:
          label[int(cnt1/2)] = label[int(cnt2/2)]
    
    else :
      row.append(0)
    
    cnt2 += 2
  
  matrix.append(row)  
  cnt1 += 2
  

이설  and  나쁜형사
이선정  and  황지만
황지만  and  이선정
나쁜형사  and  이설


In [66]:

print('\n',label,'\n')
print(matrix)

size = [0]
size = size*20

for i in range(0,20):
  size[i] = sum(matrix[i])
 
print(size)
score =[0]
score = score*20

for i in range(0,20):
  score[label[i] - 1] += 1/(i+1); #label is 1~20 => needed to subtract 1


 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 12, 13, 14, 4, 16, 17, 18, 19, 20] 

[[0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], [2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], [2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0], [2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 92, 0, 2, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 4, 0, 0], [2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], [2, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0], [2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], [2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], [2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 32, 0, 0, 0, 0, 0, 2, 0, 0, 0], [2, 0, 0, 0, 2, 0, 0, 0, 0, 110, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], [2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], [2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0], [2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], [2, 0, 0, 55, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,

In [67]:

#RESULT
print('\n☆급상승 검색어 순위☆\n')
cnt = 1;

#sorting according to SCORE
for i in range(0,20):
  max = 0
  for j in range(1,20):
    if score[j] > score[max]:
      max = j;
  
  relative_word = ''
  
  for j in range(0,20):
    if label[j] == max + 1:
      relative_word += data_list[1 + 2*j] + ' | ' 
  
  score[max] = 0;
  
  print(cnt, '. ', relative_word[0:-3])
  cnt += 1
  if(score.count(0) == 20):
    break  


☆급상승 검색어 순위☆

1 .  백석역
2 .  이일재
3 .  마녀공장
4 .  이설 | 나쁜형사
5 .  수능 성적표
6 .  이선정 | 황지만
7 .  최유정 변호사
8 .  2019 수능 만점자
9 .  광주형 일자리
10 .  pmc 더벙커
11 .  이강인
12 .  블레이드앤소울 레볼루션
13 .  다나
14 .  모털 엔진
15 .  홍현희
16 .  한국교육과정평가원
17 .  삼성전자 특별보너스
18 .  기흥 롯데프리미엄 아울렛
